# Autoencoder (AE) Implementation with Nature-Inspired Optimization
### MSC/DSA/134

This notebook implements an Autoencoder for fraud detection. 
Optimization Goal: Find the best architecture (Encoder Layers, Decoder Layers, Latent Size, Units, Dropout) that maximizes the F1 score (Anomaly Detection performance).

In [1]:
from globals.pandas_functions import *
import globals.hyperparameter_optimizer as hyp_optimizer
import globals.torch_gpu_processing as torch_gpu_processing
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
# import datasets
data_base_path = "data/processed/null_value_option_1_with_validation_set/scaled_only"

X_train = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_x_train_scaled.csv")
X_validation = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_x_validation_scaled.csv")
X_test = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_x_test_scaled.csv")

y_train = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_y_train.csv")
y_validation = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_y_validation.csv")
y_test = pd.read_csv(f"{data_base_path}/unified_transaction_data_option2_y_test.csv")

print("X_train:", X_train.shape)
print("X_validation:", X_validation.shape)
print("X_test:", X_test.shape)

X_train: (354305, 26)
X_validation: (118102, 26)
X_test: (118102, 26)


In [3]:
torch_gpu_processing.test_direct_ml_processing()

torch: 2.4.1+cpu
DirectML device: privateuseone:0
Tensor device: privateuseone:0


In [4]:
# get a sample for optimization
sample_size = 25000
seed = 42

def get_stratified_sample(X, y, sample_size, random_state=42):
    if sample_size >= len(X): return X, y
    X_sample, _, y_sample, _ = train_test_split(
        X, y, train_size=sample_size, stratify=y, random_state=random_state
    )
    return X_sample, y_sample

X_train_sample, y_train_sample = get_stratified_sample(
    X_train.to_numpy(),
    y_train.to_numpy().ravel(),
    sample_size=sample_size,
    random_state=seed
)

print(f"Optimization Sample Size: {len(X_train_sample)}")

Optimization Sample Size: 25000


In [5]:
# ==========================================
# HYPERPARAMETER OPTIMIZATION (AE)
# ==========================================

# Settings
param_optimizer_algorithm = "PSO" # (FA, GWO, etc)
population = 10
iterations = 8
epochs_for_evaluation = 6
batch_size = 2048
early_stopping = 3


from mealpy.swarm_based import FA, GWO, PSO
from mealpy.utils.space import IntegerVar, FloatVar, BoolVar, CategoricalVar

# define objective
objective_function = torch_gpu_processing.set_ae_optimizer_objective(
    X_train_sample,
    y_train_sample,
    X_validation.to_numpy(), # Pass full validation, helper handles downsampling
    y_validation.to_numpy(),
    max_epochs=epochs_for_evaluation,
    batch_size=batch_size,
    seed=seed,
    early_stopping_patience=early_stopping
)

# 2. Define Bounds (AE Specific)
bounds_cfg = hyp_optimizer.get_ae_hyperparameter_bounds_config(min_layers=1, max_layers=4)
optimizer_bounds = []
for cfg in bounds_cfg:
    if cfg['type'] == 'int':
        optimizer_bounds.append(IntegerVar(lb=cfg['lb'], ub=cfg['ub'], name=cfg.get('name')))
    elif cfg['type'] == 'float':
        optimizer_bounds.append(FloatVar(lb=cfg['lb'], ub=cfg['ub'], name=cfg.get('name')))
    elif cfg['type'] == 'bool':
        optimizer_bounds.append(BoolVar(name=cfg.get('name')))
    elif cfg['type'] == 'categorical':
        optimizer_bounds.append(CategoricalVar(valid_sets=cfg['choices'], name=cfg.get('name')))

problem = dict(obj_func=objective_function, bounds=optimizer_bounds, minmax="min", log_to=None)

# 3. Run Optimizer
print(f"Starting AE Optimization using {param_optimizer_algorithm}...")
optimizer = FA.OriginalFA(epoch=iterations, pop_size=population)

best_agent = optimizer.solve(problem)
best_vec = best_agent.solution
best_obj = best_agent.target.fitness
best_hp = hyp_optimizer.optimizer_vectors_to_ae_hyperparams(best_vec)

print("\n=== BEST SOLUTION ===")
print(f"Best Objective (1-F1): {best_obj:.6f} => F1: {1-best_obj:.6f}")
print(f"Best Hyperparameters: {best_hp}")

Optimizer using CPU for stability on small samples.
Starting AE Optimization using PSO...
[1:0.3s][2:0.2s][3:0.2s][4:0.2s][5:0.3s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.2s][5:0.1s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.2s][5:0.1s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.2s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.1s][3:0.1s][4:0.2s][5:0.1s][6:0.1s]]
[1:0.1s][2:0.2s][3:0.2s][4:0.2s][5:0.1s][6:0.1s]]
[1:0.1s][2:0.1s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.1s][2:0.1s][3:0.2s][4:0.2s][5:0.1s][6:0.1s]]
[1:0.1s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2:0.2s][3:0.2s][4:0.3s][5:0.2s][6:0.2s]]
[1:0.2s][2

In [ ]:
# ==========================================
# FINAL MODEL TRAINING
# ==========================================

model, metrics = torch_gpu_processing.train_final_ae_model(
    best_hp,
    X_train.to_numpy(),
    y_train.to_numpy(),
    X_test.to_numpy(),
    y_test.to_numpy(),
    batch_size=batch_size,
    max_epochs=50
)

print("Final Evaluation (Test Set):", metrics)